In [ ]:
# from pl_bolts.models.autoencoders import AE, VAE
from basic_ae_module import AE
from basic_vae_module import VAE
EPOCHS = 2000
IM_SIZE = 64
BATCH_SIZE = 64
ROWS = 8
# ae = AE(input_height=96, enc_type="resnet50", enc_out_dim=2048, latent_dim=256) #, kl_coeff=1.0)
ae = VAE(
    input_height=IM_SIZE, 
    enc_type="resnet18", 
    enc_out_dim=512, 
    # enc_type="custom", 
    # enc_out_dim=512, 
    latent_dim=512,
    kl_coeff=0.1,
    # kl_warmup=EPOCHS//5,
    lr=1e-6) #, kl_coeff=1.0)
# print(VAE.pretrained_weights_available())
# ae = ae.from_pretrained('cifar10-resnet18')

# ae.freeze()

In [ ]:
# from torchvision.datasets import CIFAR10, CIFAR100, MNIST, CelebA
# ds = CelebA(root='/data')

In [ ]:
from data import sprites
from torchvision.datasets import CelebA, LFWPairs, MNIST
import torchvision
import torch
train, val  = sprites.get_loader(
    batch_size=BATCH_SIZE, 
    workers=16, 
    val_ratio=0.1,
    resize_shape=(IM_SIZE, IM_SIZE))
# data = CelebA(
#         root="./data", 
#         # root="/data/torch/", 
#         download=False, 
#         transform=torchvision.transforms.Compose([
#             torchvision.transforms.Resize((64, 64)),
#             torchvision.transforms.RandomHorizontalFlip(),
#             torchvision.transforms.ToTensor(),
#             torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
#         ]))
# data = MNIST(
#         root="./data", 
#         # root="/data/torch/", 
#         download=True, 
#         transform=torchvision.transforms.Compose([
#             torchvision.transforms.Resize((64, 64)),
#             # torchvision.transforms.RandomHorizontalFlip(),
#             torchvision.transforms.ToTensor(),
#             # torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
#             torchvision.transforms.Normalize((0.5,), (0.5,)),
#         ]))
# val_count = int(len(data) * 0.1)
# train, val = torch.utils.data.random_split(data, [len(data)-val_count, val_count])
# train = torch.utils.data.DataLoader(
#     train, 
#     batch_size=BATCH_SIZE, 
#     pin_memory=True,
#     persistent_workers=True,
#     shuffle=True, 
#     num_workers=24)
# val = torch.utils.data.DataLoader(
#     val, 
#     batch_size=BATCH_SIZE, 
#     pin_memory=True,
#     persistent_workers=True,
#     shuffle=False, 
#     num_workers=12)


In [ ]:
item, label = (next(iter(train)))
import numpy as np
print(np.max(item.numpy()))
print(np.min(item.numpy()))


In [ ]:

item, label = (next(iter(train)))
item.shape

In [ ]:
len(train)

In [ ]:
import torch
from pytorch_lightning.callbacks import BasePredictionWriter, StochasticWeightAveraging, Callback, EarlyStopping
import os
import os
from typing import Any, List
import utils
import torchvision

class CustomWriter(Callback):

    def __init__(
        self,
        data: Any,
        write_interval: int = 20,
        num_rows: int = 1,
        normalize: bool = False,
        post_fix: str = "",
    ):
        super().__init__()
        self.write_interval = write_interval
        self.data = data
        self.num_rows = num_rows
        self.normalize = normalize
        self.post_fix = post_fix

    def on_epoch_end(self, trainer: "pl.Trainer", ae_module: "pl.LightningModule") -> None:

        # print("writing batch")
        # torch.save(prediction, os.path.join(
        #     self.output_dir, dataloader_idx, f"{batch_idx}.pt"))

        if (trainer.current_epoch + 1) % self.write_interval == 0:
            ae_module.eval()
            with torch.no_grad():
                results = ae_module.encode_decode(self.data.to(ae_module.device))


            grid_og = torchvision.utils.make_grid(self.data, nrow=self.num_rows, normalize=self.normalize)
            grid = torchvision.utils.make_grid(results, nrow=self.num_rows, normalize=self.normalize)
            str_title = f"{ae_module.__class__.__name__}_encode_decode{self.post_fix}"
            str_title_og = f"{ae_module.__class__.__name__}_og{self.post_fix}"
            # trainer.logger.experiment.add_image(str_title, grid, global_step=trainer.global_step)
            trainer.logger.log_image(key=str_title, images=[grid], step=trainer.global_step)
            trainer.logger.log_image(key=str_title_og, images=[grid_og], step=trainer.global_step)
        # utils.save_image(
        #     results, 
        #     os.path.join(self.output_dir, "preds"), 
        #     # convert_func=sprites.denormalize
        #     )
    # def write_on_epoch_end(
    #     self, trainer, pl_module: 'LightningModule', predictions: List[Any], batch_indices: List[Any]
    # ):
# TODO why did switching from mse to binary crossentropy make it use so much less memory???


In [ ]:
print(train)
# from pl_bolts.callbacks import LatentDimInterpolator, TensorboardGenerativeModelImageSampler
from utils import WandbGenerativeModelImageSampler, LatentDimInterpolator
from pytorch_lightning import Trainer
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks import LearningRateMonitor


tb_logger = pl_loggers.TensorBoardLogger(save_dir=f"{ae.__class__.__name__}_logs/")
wandb_logger = pl_loggers.WandbLogger(
    project="pokegen-vae"
)
trainer = Trainer(
    max_epochs=EPOCHS, 
    accelerator='auto', 
    devices=-1, 
    strategy='dp',
    # precision=16,
    # logger=tb_logger,
    logger=wandb_logger,
    # gradient_clip_val=1.0,
    # detect_anomaly=True,
    callbacks=[
        WandbGenerativeModelImageSampler(
            interpolate_epoch_interval=4,
            num_samples=BATCH_SIZE, nrow=ROWS, normalize=True, ),
        LatentDimInterpolator(
            interpolate_epoch_interval=4, 
            # range_start=-10, range_end=10, 
            ),
        CustomWriter(
            write_interval=4, 
            data=next(iter(train))[0], 
            num_rows=ROWS, 
            normalize=True,
            post_fix="_train",),
        CustomWriter(
            write_interval=4, 
            data=next(iter(val))[0], 
            num_rows=ROWS, 
            normalize=True,
            post_fix="_val",),
        LearningRateMonitor(),
        EarlyStopping(monitor="val_loss", patience=150),
        ],
        # CustomWriter('preds', 'epoch'),
        # CustomWriter('preds', 'epoch'),
)
# trainer.predict(ae, sprites.PokeModule())
# trainer.predict(ae, train)
trainer.fit(ae, train_dataloaders=train, val_dataloaders=val)
# TODO AE loss function with dicriminator for which is actual vs encoded

In [ ]:
# r = trainer.predict(ae, dataloaders=train)